# Data Processing
Notebook for handling the processing of our datasets.

In [1]:
# depenencies
import numpy as np
import pandas

In [2]:
# read in datasets
verifications_2015 = pandas.read_csv("2015_pos_verified_tweets_cleaned.csv", encoding='latin-1', usecols=["st_y", "st_x", "location_country", "created_at"])
verifications_2016 = pandas.read_csv("2016_pos_verified_tweets_cleaned.csv", encoding='latin-1', usecols=["st_y", "st_x", "location_country", "created_at"])
flare_data = pandas.read_csv("flares_and_instruments_v2.csv", encoding='latin-1')

In [3]:
# fixing date-time formats 
# flare data has (YYYY-MM-DD)'T'(HH:MM:SS)
# verification data has (YYYY-MM-DD) (HH:MM:SS)'+00'

# make one big dataframe from the 2 verification dataframes
verifications = pandas.concat([verifications_2015, verifications_2016], ignore_index=True)

# remove '+00' from date-time in verification data
f = lambda x: x[:-3]
verifications['created_at'] = verifications['created_at'].apply(f)

# replace 'T' with ' ' from date-time in flare data
g = lambda x: x.replace("T", " ")
flare_data['JJJ Start'] = flare_data['JJJ Start'].apply(g)
flare_data['JJJ Peak'] = flare_data['JJJ Peak'].apply(g)
flare_data['JJJ End'] = flare_data['JJJ End'].apply(g)

# reduce Sol to only be the date in format YYYYMMDD
h = lambda x: x[:8]
flare_data['Sol'] = flare_data['Sol'].apply(h)

# add Sol column to verifications
verifications['created_at'] = pandas.to_datetime(verifications['created_at'], errors='coerce')
if verifications['created_at'].isnull().any():
    print("Warning: Some rows have invalid datetime values!")
verifications['Sol'] = verifications['created_at'].dt.strftime('%Y%m%d')

In [4]:
# group verifications by date
verifications_unique = verifications.groupby('Sol').agg(
    {col: lambda x: tuple(x) for col in verifications.columns if col != 'Sol'}
).reset_index()

# add a column for the number of verifications per date
verifications_unique['sighting_count'] = verifications_unique['created_at'].apply(len)

# group flare_data by date
flare_data_unique = flare_data.groupby('Sol').agg(
    {col: lambda x: tuple(x) for col in flare_data.columns if col != 'Sol'}
).reset_index()

# add a column for the number of flares per date
flare_data_unique['Flare Count'] = flare_data_unique['JJJ Class'].apply(len)

In [5]:
print(verifications_unique['sighting_count'])

0      1
1      7
2      2
3      6
4      2
      ..
232    2
233    1
234    2
235    1
236    2
Name: sighting_count, Length: 237, dtype: int64


In [6]:
# merge dataframes
aurorae = pandas.merge(flare_data_unique, verifications_unique, on='Sol', how='outer')

# Add the 'aurora_visible' column
aurorae['aurora_visible'] = np.where(aurorae['sighting_count'].notna() & (aurorae['sighting_count'] > 0), 1, 0)

# Set 'sightings_count' to 0 where 'aurora_visible' is 0
aurorae.loc[aurorae['aurora_visible'] == 0, 'sighting_count'] = 0

In [7]:
# save dataset
aurorae.to_csv('../data_processed/aurorae_dataset.csv')
np.save('../data_processed/aurorae_dataset', aurorae)

In [10]:
print(aurorae.columns)
print(aurorae.dtypes)

Index(['Sol', 'JJJ Start', 'JJJ Peak', 'JJJ End', 'JJJ Class', 'HHH X-pos',
       'HHH y-pos', 'AAA lo', 'AAA hi', 'AAA X-pos', 'AAA Y-pos', 'AAA', 'BBB',
       'CCC', 'DDD', 'EEE', 'FFF', 'GGG', 'Flare Count', 'st_y', 'st_x',
       'location_country', 'created_at', 'sighting_count', 'aurora_visible'],
      dtype='object')
Sol                  object
JJJ Start            object
JJJ Peak             object
JJJ End              object
JJJ Class            object
HHH X-pos            object
HHH y-pos            object
AAA lo               object
AAA hi               object
AAA X-pos            object
AAA Y-pos            object
AAA                  object
BBB                  object
CCC                  object
DDD                  object
EEE                  object
FFF                  object
GGG                  object
Flare Count         float64
st_y                 object
st_x                 object
location_country     object
created_at           object
sighting_count      float64